In [201]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split

In [257]:
data = pd.read_csv('train.csv')

In [258]:
data.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644


In [259]:
categorical_columns = [c for c in data.columns if data[c].dtype.name == 'object']
numerical_columns   = [c for c in data.columns if data[c].dtype.name != 'object']
print (categorical_columns)
print (numerical_columns)

['Ecology_2', 'Ecology_3', 'Shops_2']
['Id', 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Social_1', 'Social_2', 'Social_3', 'Healthcare_1', 'Helthcare_2', 'Shops_1', 'Price']


In [260]:
data.count(axis=0)

Id               10000
DistrictId       10000
Rooms            10000
Square           10000
LifeSquare        7887
KitchenSquare    10000
Floor            10000
HouseFloor       10000
HouseYear        10000
Ecology_1        10000
Ecology_2        10000
Ecology_3        10000
Social_1         10000
Social_2         10000
Social_3         10000
Healthcare_1      5202
Helthcare_2      10000
Shops_1          10000
Shops_2          10000
Price            10000
dtype: int64

In [261]:
data = data.fillna(data.median(axis=0), axis=0)

In [262]:
data.count(axis=0)

Id               10000
DistrictId       10000
Rooms            10000
Square           10000
LifeSquare       10000
KitchenSquare    10000
Floor            10000
HouseFloor       10000
HouseYear        10000
Ecology_1        10000
Ecology_2        10000
Ecology_3        10000
Social_1         10000
Social_2         10000
Social_3         10000
Healthcare_1     10000
Helthcare_2      10000
Shops_1          10000
Shops_2          10000
Price            10000
dtype: int64

In [263]:
data_describe = data.describe(include=[object])
data_describe
binary_columns    = [c for c in categorical_columns if data_describe[c]['unique'] == 2]
nonbinary_columns = [c for c in categorical_columns if data_describe[c]['unique'] > 2]
print (binary_columns, nonbinary_columns)

['Ecology_2', 'Ecology_3', 'Shops_2'] []


In [448]:
for c in binary_columns[1:]:
    top = data_describe[c]['top']
    top_items = data[c] == top
    data.loc[top_items, c] = 0
    data.loc[np.logical_not(top_items), c] = 1

In [267]:
data[binary_columns].describe()

,Ecology_2,Ecology_3,Shops_2
count,10000.000000,10000.000000,10000.000000
mean,0.009700,0.027500,0.082500
std,0.098015,0.163543,0.275139
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000


In [268]:
data_numerical = data[numerical_columns]
data_numerical = (data_numerical - data_numerical.mean()) / data_numerical.std()
data_numerical.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04
mean,5.773160e-17,-7.354117e-17,-5.808687e-17,4.078737e-16,-1.084850e-14,1.151010e-16,3.501643e-17,-2.519984e-16,-2.039558e-16,-3.586826e-14,-6.164513e-18,-4.662104e-17,1.285211e-15,8.626433e-17,1.193456e-15,-3.351763e-16,-7.169175e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.725329e+00,-1.156311e+00,-2.251904e+00,-2.620239e+00,-4.685476e-01,-2.196463e-01,-1.436078e+00,-1.860899e+00,-1.037488e-02,-9.985907e-01,-1.408062e+00,-1.293840e+00,-3.373297e-01,-1.374595e+00,-8.834356e-01,-8.803579e-01,-1.668572e+00
25%,-8.672342e-01,-6.974645e-01,-1.060735e+00,-6.904924e-01,-1.401729e-01,-1.846334e-01,-8.636848e-01,-5.326762e-01,-1.005568e-02,-8.503276e-01,-1.065842e+00,-9.454322e-01,-3.373297e-01,-2.629820e-01,-8.834356e-01,-6.722994e-01,-6.489150e-01
50%,2.282827e-03,-3.303876e-01,1.304329e-01,-1.805648e-01,-4.548728e-02,-9.569020e-03,-2.912911e-01,5.764485e-02,-1.004072e-02,-3.649128e-01,1.785244e-02,-1.676086e-02,-2.534085e-01,-1.692315e-01,-2.139126e-01,-2.561824e-01,-2.354762e-01
75%,8.662432e-01,5.643624e-01,1.304329e-01,4.551485e-01,6.736973e-02,9.546962e-02,6.626983e-01,6.479659e-01,-9.921016e-03,6.462791e-01,6.452546e-01,4.679152e-01,-1.275267e-01,-4.869521e-02,4.556104e-01,3.679931e-01,3.768250e-01
max,1.731747e+00,3.638632e+00,2.038029e+01,2.776755e+01,9.717175e+01,7.029630e+01,6.386635e+00,1.540599e+01,9.999000e+01,3.385916e+00,2.812644e+00,3.426885e+00,5.579116e+00,5.119635e+00,3.133702e+00,3.904987e+00,4.512590e+00


In [293]:
data = pd.concat((data_numerical, data[binary_columns]), axis=1)
data = pd.DataFrame(data, dtype=float)
print (data.shape)
print (data.columns)

(10000, 20)
Index(['Id', 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare',
       'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Social_1', 'Social_2',
       'Social_3', 'Healthcare_1', 'Helthcare_2', 'Shops_1', 'Price',
       'Ecology_2', 'Ecology_3', 'Shops_2'],
      dtype='object')


In [574]:
fts = ['Square', 'Rooms','Square','HouseFloor','DistrictId', 'Floor','Shops_1', 'Ecology_1',
       'Ecology_2', 'Ecology_3', 'Shops_2','Social_1', 'Social_2',
       'Social_3']
X = data.loc[:, fts]
y = data['Price']

In [575]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=11)

In [576]:
from sklearn.ensemble import RandomForestRegressor as RF

rf = RF(n_estimators=20, max_depth=10, min_samples_leaf=2, random_state=42)

In [577]:
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [578]:
pred = rf.predict(X_train)

In [579]:
pred

array([ 0.64101026,  0.22650757, -0.26596805, ..., -0.56189074,
       -0.70688438, -0.14573929])

In [580]:
r2(y_train, pred)

0.8241818681347406

In [581]:
pred_valid = rf.predict(X_test)

In [582]:
pred_valid

array([-0.5827365 , -0.70189377, -0.36210718, ...,  0.18321323,
       -0.53748761, -0.49240791])

In [583]:
r2(y_test, pred_valid)

0.7327922160543129

## Предсказания на тесте

In [523]:
test = pd.read_csv('test.csv')

In [524]:
test.shape

(5000, 19)

In [525]:
test.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
0,725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,B,B,11,2748,1,NaN,0,0,B
1,15856,74,2.0,69.263183,NaN,1.0,6,1.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B
2,5480,190,1.0,13.597819,15.948246,12.0,2,5.0,1909,0.000000,B,B,30,7538,87,4702.0,5,5,B
3,15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,B,B,23,4583,3,NaN,3,3,B
4,14275,27,1.0,47.527111,43.387569,1.0,17,17.0,2017,0.072158,B,B,2,629,1,NaN,0,0,A


In [526]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 19 columns):
Id               5000 non-null int64
DistrictId       5000 non-null int64
Rooms            5000 non-null float64
Square           5000 non-null float64
LifeSquare       3959 non-null float64
KitchenSquare    5000 non-null float64
Floor            5000 non-null int64
HouseFloor       5000 non-null float64
HouseYear        5000 non-null int64
Ecology_1        5000 non-null float64
Ecology_2        5000 non-null object
Ecology_3        5000 non-null object
Social_1         5000 non-null int64
Social_2         5000 non-null int64
Social_3         5000 non-null int64
Healthcare_1     2623 non-null float64
Helthcare_2      5000 non-null int64
Shops_1          5000 non-null int64
Shops_2          5000 non-null object
dtypes: float64(7), int64(9), object(3)
memory usage: 742.3+ KB


In [546]:
tcategorical_columns = [c for c in test.columns if test[c].dtype.name == 'object']
tnumerical_columns   = [c for c in test.columns if test[c].dtype.name != 'object']
print (tcategorical_columns)
print (tnumerical_columns)

['Ecology_2', 'Ecology_3', 'Shops_2']
['Id', 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Social_1', 'Social_2', 'Social_3', 'Healthcare_1', 'Helthcare_2', 'Shops_1', 'Price']


In [547]:
test_describe = test.describe(include=[object])
test_describe
tbinary_columns    = [c for c in tcategorical_columns if test_describe[c]['unique'] == 2]
tnonbinary_columns = [c for c in tcategorical_columns if test_describe[c]['unique'] > 2]
print (tbinary_columns, tnonbinary_columns)

['Ecology_2', 'Ecology_3', 'Shops_2'] []


In [553]:
for c in tbinary_columns[:1]:
    top = test_describe[c]['top']
    top_items = test[c] == top
    test.loc[top_items, c] = 0
    test.loc[np.logical_not(top_items), c] = 1

In [554]:
test[tbinary_columns].describe()

,Ecology_2,Ecology_3,Shops_2
count,5000.000000,5000.000000,5000.000000
mean,0.009600,0.029800,0.082400
std,0.097518,0.170052,0.275001
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000


In [555]:
test.count(axis=0)

Id               5000
DistrictId       5000
Rooms            5000
Square           5000
LifeSquare       5000
KitchenSquare    5000
Floor            5000
HouseFloor       5000
HouseYear        5000
Ecology_1        5000
Ecology_2        5000
Ecology_3        5000
Social_1         5000
Social_2         5000
Social_3         5000
Healthcare_1     5000
Helthcare_2      5000
Shops_1          5000
Shops_2          5000
Price            5000
dtype: int64

In [556]:
test = test.fillna(data.median(axis=0), axis=0)

In [557]:
test.count(axis=0)

Id               5000
DistrictId       5000
Rooms            5000
Square           5000
LifeSquare       5000
KitchenSquare    5000
Floor            5000
HouseFloor       5000
HouseYear        5000
Ecology_1        5000
Ecology_2        5000
Ecology_3        5000
Social_1         5000
Social_2         5000
Social_3         5000
Healthcare_1     5000
Helthcare_2      5000
Shops_1          5000
Shops_2          5000
Price            5000
dtype: int64

In [558]:
test_numerical = test[tnumerical_columns]
test_numerical = (test_numerical - test_numerical.mean()) / test_numerical.std()
test_numerical.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03
mean,-2.919887e-17,-3.695932e-17,6.830647e-17,4.378004e-15,1.771048e-14,-8.111289e-17,6.433742e-17,-1.233236e-16,5.486611e-15,4.907785e-15,-1.227352e-16,-1.509459e-16,-2.783551e-16,1.595286e-16,-1.409717e-15,2.756018e-16,-2.651104e-14
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.740568e+00,-1.160702e+00,-2.277621e+00,-2.884385e+00,-2.208690e+00,-6.006823e-01,-1.391881e+00,-1.856032e+00,-4.113067e+00,-9.983686e-01,-1.422172e+00,-1.301068e+00,-3.462405e-01,-1.342816e+00,-8.915228e-01,-8.881047e-01,-2.138507e+00
25%,-8.671898e-01,-6.853682e-01,-1.085149e+00,-7.617154e-01,-6.054156e-01,-5.001800e-01,-8.447579e-01,-5.304002e-01,-6.133909e-01,-8.358898e-01,-1.079944e+00,-9.543749e-01,-3.462405e-01,-2.861959e-01,-8.915228e-01,-6.787843e-01,-4.077526e-01
50%,-1.905682e-02,-3.232090e-01,1.073224e-01,-1.847902e-01,-1.608801e-01,2.331654e-03,-2.976349e-01,-8.852277e-02,-3.980262e-01,-3.974049e-01,3.775909e-03,-3.027357e-02,-2.624314e-01,-1.687936e-01,-2.158196e-01,-2.601434e-01,-1.062530e-01
75%,8.661156e-01,5.821890e-01,1.073224e-01,5.151489e-01,3.948377e-01,3.038386e-01,6.142367e-01,6.479396e-01,8.403206e-01,6.321919e-01,6.311928e-01,4.669183e-01,-1.367178e-01,-5.139140e-02,4.598836e-01,3.678179e-01,6.996465e-01
max,1.734527e+00,3.637907e+00,1.799440e+01,8.746978e+00,1.681305e+01,6.171076e+01,1.265094e+01,1.272592e+01,1.917144e+00,3.347996e+00,2.798633e+00,3.396426e+00,5.562300e+00,4.982556e+00,3.162696e+00,3.926265e+00,4.900124e+00


In [568]:
test = pd.concat((test_numerical, test[tbinary_columns]), axis=1)
test = pd.DataFrame(test, dtype=float)
print (test.shape)
print (test.columns)

(5000, 20)
Index(['Id', 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare',
       'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Social_1', 'Social_2',
       'Social_3', 'Healthcare_1', 'Helthcare_2', 'Shops_1', 'Price',
       'Ecology_2', 'Ecology_3', 'Shops_2'],
      dtype='object')


In [584]:
pred_test = rf.predict(test.loc[:, fts])
pred_test

array([-0.75009135,  0.36131817,  2.15368183, ...,  1.3197813 ,
       -0.02049804, -0.15581557])

In [585]:
pred_test.shape

(5000,)

In [586]:
test['Price'] = pred_test

In [588]:
test.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price,Ecology_2,Ecology_3,Shops_2
0,-1.590754,0.152125,0.107322,-0.343944,-0.128980,0.002332,-0.480009,0.206062,-0.667232,1.585122,-0.794755,-0.660331,-0.304336,-0.168794,-0.891523,-0.888105,-0.750091,0.0,0.0,0.0
1,1.540225,0.514284,0.107322,0.671127,-0.160880,-0.500180,-0.480009,-1.708740,-0.398026,-0.367246,-1.079944,-0.985915,-0.220527,-0.168794,-0.891523,-0.469464,0.361318,0.0,0.0,0.0
2,-0.606827,3.139938,-1.085149,-2.244391,-1.227576,0.605346,-1.209507,-1.119570,-4.059226,-0.998369,0.288965,0.529254,3.299455,4.790799,2.486993,0.158497,2.153682,0.0,0.0,0.0
3,1.500495,-0.096859,0.107322,0.869287,1.033926,0.303839,2.437980,1.384402,1.217209,-0.149925,-0.110300,-0.204614,-0.220527,-0.168794,1.135587,-0.260143,1.611686,0.0,0.0,0.0
4,1.213077,-0.549558,-1.085149,-0.467317,0.496503,-0.500180,1.526108,0.647940,1.755620,-0.397405,-1.308096,-1.186580,-0.304336,-0.168794,-0.891523,-0.888105,-0.869087,0.0,0.0,1.0


In [589]:
test.loc[:, ['Id', 'Price']].to_csv('EReukova_predictions.csv', index=None)